<a href="https://colab.research.google.com/github/tekliyetamiru/Natural-Language-Processing-NLP-with-TensorFlow/blob/main/Natural_Language_Processing_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction of NLP Fundamentals in TensorFlow
NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problem is sequence to sequence problems(seq2seq)

# Check for GPU

In [ ]:
!nvidia-smi -L

## Get helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves, compare_historys

--2025-11-05 17:22:56--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-11-05 17:22:56 (37.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text Dataset

The dataset we're going to be using is kaggle's introduction to NLP dataset ( text samples of Tweets labelled as disaster or not disaster).

See the orginal here: https://www.kaggle.com/c/nlp-getting-started

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-11-05 17:23:10--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.189.207, 142.250.152.207, 209.85.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.189.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2025-11-05 17:23:10 (88.8 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

To visualize out text samples,we first have to read them in, one way to do so would be to use python: https://realpython.com/read-write-files-python/

But I prefer to get visual straight way.
So another way to do this is to use pandas...

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
# shuffle training dataframe
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# what does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# how many examples of each class?
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [ ]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
# Let's visualize some random training examples
import random
random_index=random.randint(0,len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","(real diaster)" if target>0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target:1 (real diaster)
Text:
#Earthquake #Sismo M 1.4 - 4km E of Interlaken California: Time2015-08-06 00:52:25 UTC2015-08-05 17:52:25 -07... http://t.co/wA5C77F8vQ

---

Target:0 (not real diaster)
Text:
Being able to stay out of work this week to take online courses for the Property and Casualty StateÛ_ https://t.co/jmD7zwKSDM

---

Target:1 (real diaster)
Text:
Remembering Marlene Menahem 22 of Moshav Safsufa;  murdered by Hamas terrorists in the suicide bombing of Egged bus No. 361

---

Target:0 (not real diaster)
Text:
Speaking of memorable debates: 60-Second Know-It-All: Ronald Reagan's landslide romp on Election Day in 1980 m... http://t.co/2XOhtjQJWh

---

Target:0 (not real diaster)
Text:
I just heard a really loud bang and everyone is asleep great

---



### Split data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Use train_test_split to split training data into training and validation sets
train_sentences,val_sentences,train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                          train_df_shuffled["target"].to_numpy(),
                                                                          test_size=0.1, #use 10% of training data for valid
                                                                          random_state=42)


In [ ]:
# Check the lengths
len(train_sentences), len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [ ]:
# Check the first 10 samples
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of teature vector for each token(this size of the features vector can be defined and this embedding can be learned)


### Text Vectorization(tokenization)

In [ ]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [ ]:
from os.path import split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization


# Use the default TextVectorization parameters

text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # Create groups of n-words?
                                    output_mode="int", # how to map token to numbers
                                    output_sequence_length=15, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)



In [ ]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [ ]:
# Find the average number of token (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
for i in train_sentences[:5]:
  print(i.split(),"\n")


['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'] 

['Imagine', 'getting', 'flattened', 'by', 'Kurt', 'Zouma'] 

['@Gurmeetramrahim', '#MSGDoing111WelfareWorks', 'Green', 'S', 'welfare', 'force', 'ke', 'appx', '65000', 'members', 'har', 'time', 'disaster', 'victim', 'ki', 'help', 'ke', 'liye', 'tyar', 'hai....'] 

['@shakjn', '@C7', '@Magnums', 'im', 'shaking', 'in', 'fear', "he's", 'gonna', 'hack', 'the', 'planet'] 

['Somehow', 'find', 'you', 'and', 'I', 'collide', 'http://t.co/Ee8RpOahPk'] 



In [ ]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vacabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a tweet does a model see?)

text_vectorizer=TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)



In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentence and tokenize it
sample_sentence= "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence=random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \\n\nVectorized Version:")
text_vectorizer([random_sentence])

Original text:
 After a Few Years Afloat Pension Plans Start Sinking Again http://t.co/4cEEuzWHvf \n
Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  43,    3,  548,  141,    1, 9862,  645,  667,  253,  282,    1,
           0,    0,    0,    0]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words

print(f"Number of words in vocab:{len(words_in_vocab)}")
print(f"5 most common words:{top_5_words}")
print(f"5 least common words:{bottom_5_words}")

Number of words in vocab:10000
5 most common words:['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
5 least common words:[np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


## Creating an Embedding using an Embedding Layer

To make out embedding, we're going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding


The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer


In [ ]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape
                             input_length=max_length # how long is each input
                             )
embedding

<Embedding name=embedding_3, built=False>

In [ ]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 @POTUS you until you drown by water entering the lungs. You being alive has caused this great country to fall to shit because you're a pussy      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03786952, -0.02512523, -0.03903513, ..., -0.04990149,
          0.04297302, -0.04947592],
        [-0.01729473, -0.00540066, -0.01355418, ...,  0.02241972,
         -0.04758586,  0.01123397],
        [-0.01589712,  0.01954892, -0.04190894, ..., -0.02365153,
          0.01286652, -0.0194711 ],
        ...,
        [ 0.03477874,  0.03012707, -0.04173044, ..., -0.04088559,
         -0.03491502, -0.0472248 ],
        [ 0.0441232 ,  0.04373181,  0.04235106, ..., -0.04151362,
         -0.00850831,  0.01102011],
        [ 0.0456867 , -0.03965565, -0.03949145, ..., -0.02166926,
          0.02198174, -0.03461186]]], dtype=float32)>

In [ ]:
# check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]


(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02531172, -0.02525865,  0.00536426,  0.04706872,  0.01711266,
         0.03688803,  0.02721036,  0.03021869, -0.00944873,  0.0465898 ,
        -0.02787547,  0.01594844,  0.0125686 ,  0.00660314, -0.03622771,
         0.00089711,  0.03141946,  0.00365814,  0.04938667,  0.02433432,
        -0.0382819 , -0.001107  ,  0.01401604,  0.01976905,  0.03216025,
        -0.0385061 , -0.01945237,  0.04991465, -0.02813645,  0.04322103,
         0.03999704,  0.01976432,  0.04191506, -0.00694935,  0.02863609,
         0.04480517,  0.02033171,  0.0026342 ,  0.03104378,  0.04412848,
        -0.02423712, -0.00238059,  0.0345795 , -0.01623899, -0.03691473,
        -0.0173304 , -0.00363662, -0.0370348 , -0.018136  , -0.04132351,
         0.03045212,  0.04845972, -0.00829172, -0.01581477,  0.03194353,
        -0.04621202, -0.03302525,  0.03401392,  0.00446843,  0.00800122,
         0.00678091, -0.03693886,  0.01284338, -0.02306124,  0.00576542,
  

## Modelling a text dataset (running a series of experiments)

Now we've a got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from sklearn ML
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM mode (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to apporach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate our model



### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> 🔑 **Note:** Is's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([("tfidf",TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf",MultinomialNB()) # model the text
                    ])

# Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}% ")

Our baseline model achieves an accuracy of: 79.27% 


In [ ]:
# make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function  for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however, this will be cumbersome and could easily be fixed with a function.

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score


In [ ]:
# Function to evaluate :accuracy,precision,recall,f1-score
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  """
  Calculates mode accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy=accuracy_score(y_true,y_pred)*100
  # Calculate model precision, recall and f1-score using "weighted" average
  model_predicison, model_recall, model_f1, _ = precision_recall_fscore_support(y_true,y_pred,average="weighted")
  model_results = {"accuracy" : model_accuracy,
                   "precision" : model_predicison,
                   "recall": model_recall,
                   "f1" : model_f1}

  return model_results

In [ ]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [ ]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype=tf.string) # input are 1-dimensional strings
x=text_vectorizer(inputs) # turn the input text into numbers
x=embedding(x) # create an embedding of the numberized inputs
outputs = layers.Dense(1,activation="sigmoid")(x) # Create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15, 1)          │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
inputs,outputs

(<KerasTensor shape=(None, 1), dtype=string, sparse=False, ragged=False, name=keras_tensor_4>,
 <KerasTensor shape=(None, 15, 1), dtype=float32, sparse=False, ragged=False, name=keras_tensor_7>)

In [ ]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
mode_1_history= model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20251105-184252
Epoch 1/5


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 15, 1)